In [16]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Input, Dropout, BatchNormalization, Activation, ConvLSTM2D, Flatten
from keras import backend as K
from keras.models import Model
import datetime
import os

In [2]:
data_path = './data/'   

train = pd.read_csv(data_path + 'train_features.csv')
train_label = pd.read_csv(data_path + 'train_labels.csv')
test = pd.read_csv(data_path + 'test_features.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

In [3]:
x_train = []

for uid in tqdm(train['id'].unique()):
    temp = np.array(train[train['id'] == uid].iloc[:,2:], np.float32)
    x_train.append(temp)

x_train = np.array(x_train, np.float32)
x_train = x_train[:,:,:]

x_test = []

for uid in tqdm(test['id'].unique()):
    temp = np.array(test[test['id'] == uid].iloc[:,2:], np.float32)
    x_test.append(temp)

x_test = np.array(x_test, np.float32)
x_test = x_test[:,:,:]

100%|██████████| 782/782 [00:00<00:00, 1283.20it/s]


In [4]:
y = tf.keras.utils.to_categorical(train_label['label']) 
y.shape

(3125, 61)

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, y, test_size=0.3, stratify=y, random_state=42, shuffle=True)

In [6]:
for c in range(1):
    for a in tqdm(range(50)):
        plus_data = []


        for i in range(2187):
            if (y_train[i][26] != 1):
                shift_data = np.roll(X_train[i], int(random.random()*600), axis=0)
                plus_data.append(shift_data)

        plus_data_label = []

        for i in range(2187):
            if (y_train[i][26] != 1):
                shift_label = y_train[i]
                plus_data_label.append(shift_label)

        plus_data = np.array(plus_data, np.float32)
        plus_data_label = np.array(plus_data_label, np.float32)

        X_train= np.vstack([X_train, plus_data])
        y_train=np.vstack([y_train, plus_data_label])

"""    for b in range(1):
        plus_data = []

        for i in range(2187):
            if (y_train[i][26] == 1):
                shift_data = np.roll(X_train[i], int(random.random()*600), axis=0)
                plus_data.append(shift_data)

        plus_data_label = []

        for i in range(2187):
            if (y_train[i][26] == 1):
                shift_label = y_train[i]
                plus_data_label.append(shift_label)

        plus_data = np.array(plus_data, np.float32)
        plus_data_label = np.array(plus_data_label, np.float32)

        X_train= np.vstack([X_train, plus_data])
        y_train=np.vstack([y_train, plus_data_label])"""

100%|██████████| 50/50 [00:10<00:00,  4.95it/s]


'    for b in range(1):\n        plus_data = []\n\n        for i in range(2187):\n            if (y_train[i][26] == 1):\n                shift_data = np.roll(X_train[i], int(random.random()*600), axis=0)\n                plus_data.append(shift_data)\n\n        plus_data_label = []\n\n        for i in range(2187):\n            if (y_train[i][26] == 1):\n                shift_label = y_train[i]\n                plus_data_label.append(shift_label)\n\n        plus_data = np.array(plus_data, np.float32)\n        plus_data_label = np.array(plus_data_label, np.float32)\n\n        X_train= np.vstack([X_train, plus_data])\n        y_train=np.vstack([y_train, plus_data_label])'

In [7]:
def result_analysis(model_pred,label,threshold,path):
    '''
    model_pred  : 모델의 예측 
    label       : 실제 라벨
    threshold   : weak class로 구분할 기준 accuracy 값 (0~1)
    path        : 그래프 저장경로
    '''
    

    # label dictionary 생성
    train_x_scaled = pd.read_csv('./data/train_features.csv') #train feature load
    train_y = pd.read_csv('./data/train_labels.csv') #label load
    label_dict = dict()
    for label, label_desc in zip(train_y.label, train_y.label_desc):
        label_dict[label] = label_desc
    label_dict[45] ='Squat (kettlebell , goblet)'
    
    class_accuracy = dict()
    if not (path in os.listdir()):
        os.mkdir(path)
        
    for l in range(61):
        desc = label_dict[l]
        idx = label.argmax(axis = 1) == l
        class_pred = model_pred[idx]

        #클래스별 분류 확률 계산
        class_pred = np.unique(class_pred,return_counts = True)
        print((class_pred[1][np.where(class_pred[0]==l)] / class_pred[1].sum()))
        class_accuracy[desc] = (class_pred[1][np.where(class_pred[0]==l)] / class_pred[1].sum())[0]

        if class_accuracy[desc] < threshold: # weak class threshold
            #클래스별 분류 결과 그림
            plt.figure(figsize = (15,10))
            plt.title(desc,fontsize = 20)
            plt.bar([label_dict[l] for l in class_pred[0]],class_pred[1])
            plt.xticks(fontsize = 15,rotation = 90)
            plt.savefig(os.path.join('./' + path,desc), bbox_inches='tight')
            plt.clf()
            plt.close('all')
    plt.figure(figsize = (15,10))
    plt.title('Accuracy by class')
    
    weak_labels = []
    weak_accuracy = []
    for key in class_accuracy.keys():
        if class_accuracy[key] <0.8:
            weak_labels.append(key)
            weak_accuracy.append(class_accuracy[key])
    plt.bar(weak_labels,weak_accuracy)
    plt.xticks(rotation = 90)
    plt.savefig(os.path.join('./' + path,'Weakaccuracy'), bbox_inches='tight')
    plt.clf()
    plt.close('all')

In [8]:
model = Sequential()
model.add(ConvLSTM2D(filters=64, return_sequences=True, kernel_size=(1, 3), input_shape=(10, 1, 60, 6)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ConvLSTM2D(filters=64, return_sequences=True, kernel_size=(1, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ConvLSTM2D(filters=64, return_sequences=True, kernel_size=(1, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ConvLSTM2D(filters=64, return_sequences=True, kernel_size=(1, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ConvLSTM2D(filters=64, return_sequences=True, kernel_size=(1, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ConvLSTM2D(filters=64, return_sequences=True, kernel_size=(1, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(ConvLSTM2D(filters=64, kernel_size=(1, 10), activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(100, activation='relu'))
model.add(Dense(61, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
log_dir = "./Exercise_Tensorboard/Exercise_Tensorboard" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [10]:
X_train = X_train.reshape((X_train.shape[0], 10, 1, 60, 6))
X_test = X_test.reshape((X_test.shape[0], 10, 1, 60, 6))
x_test = x_test.reshape((x_test.shape[0], 10, 1, 60, 6))

In [11]:
model.fit(X_train,y_train, epochs=200, batch_size=256, shuffle=True , validation_data=(X_test, y_test), callbacks=[tensorboard_callback])

Epoch 1/200
229/229 [==============================] - 120s 524ms/step - loss: 2.5508 - accuracy: 0.2855 - val_loss: 4.0642 - val_accuracy: 0.0117
Epoch 2/200
229/229 [==============================] - 117s 513ms/step - loss: 1.0979 - accuracy: 0.6293 - val_loss: 2.2646 - val_accuracy: 0.3401
Epoch 3/200
229/229 [==============================] - 117s 511ms/step - loss: 0.6276 - accuracy: 0.7832 - val_loss: 2.6169 - val_accuracy: 0.3742
Epoch 4/200
229/229 [==============================] - 117s 511ms/step - loss: 0.3969 - accuracy: 0.8674 - val_loss: 2.7561 - val_accuracy: 0.3849
Epoch 5/200
229/229 [==============================] - 118s 515ms/step - loss: 0.2612 - accuracy: 0.9158 - val_loss: 2.5889 - val_accuracy: 0.4350
Epoch 6/200
229/229 [==============================] - 117s 513ms/step - loss: 0.2178 - accuracy: 0.9327 - val_loss: 2.5594 - val_accuracy: 0.4414
Epoch 7/200
229/229 [==============================] - 117s 509ms/step - loss: 0.1806 - accuracy: 0.9464 - val_loss: 2

KeyboardInterrupt: 

In [12]:
print(model.evaluate(X_test, y_test, batch_size=128))

8/8 [==============================] - 1s 73ms/step - loss: 2.4465 - accuracy: 0.5800
[2.446502685546875, 0.5799573659896851]


In [18]:
prediction=model.predict(x_test)
label = "loaded label"
folder_dir = "folder_directory" #미리 만들지 않아도 자동으로 폴더가 생성됩니다.
result_analysis(prediction, label, 0.7, folder_dir)
submission.iloc[:,1:]=prediction

AttributeError: 'int' object has no attribute 'argmax'

In [49]:
submission.to_csv('./output/PlusData_CNNLSTM_210212_01.csv', index=False)